In [1]:
%config IPCompleter.greedy=True
from IPython.display import IFrame

import pandas as pd
import numpy as np
import string
import math

import scipy.stats as sts

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn import preprocessing as prep
import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
from sklearn import discriminant_analysis as disan
from sklearn import calibration as calib
from sklearn import linear_model as lm
from sklearn import svm
from sklearn import gaussian_process as gaup
from sklearn import mixture as mix
from sklearn import tree
from sklearn import ensemble as ens

import tensorflow as tf
from tensorflow import keras

# from keras import models as kermdls
# from keras import layers as kerlrs
# from keras import metrics as kmetrics

from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe

import pickle

import nilearn as nl
from nilearn import plotting, image
from nilearn import datasets
import nibabel as nb
import h5py

Using TensorFlow backend.


In [2]:
data = pd.read_csv('00_Data/train_scores.csv')
data

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641
3,10005,66.532630,NaN,NaN,52.108977,69.993075
4,10007,38.617381,49.197021,65.674285,40.151376,34.096421
...,...,...,...,...,...,...
5872,21746,14.257265,21.358872,61.165998,51.778483,54.640179
5873,21747,55.456978,68.169675,29.907995,55.349257,54.019517
5874,21750,48.948756,55.114811,60.878271,38.617246,50.679885
5875,21752,66.532630,59.844808,72.303110,55.458281,46.870235


In [3]:
data.isnull().sum()

Id                0
age               0
domain1_var1    438
domain1_var2    438
domain2_var1     39
domain2_var2     39
dtype: int64

In [4]:
# def load_individual_maps_img(ids, icn_idx, ntwk_idx):
#     maps = {}
#     for patient_id in ids:
#         patient_SM = h5py.File('00_Data/fMRI_train/{0}.mat'.format(patient_id), mode='r')
#         patient_SM = np.array(patient_SM.get('SM_feature'))
    
#         patient_maps = {}
#         for i in icn_idx.keys():
#             component_maps = {}
#             for j, _ in enumerate(icn_idx[i]):
#                 jj = ntwk_idx[i][j]
#                 component_maps[jj] = patient_SM[icn_idx[i]][j].transpose([2, 1, 0])
#             patient_maps[i] = component_maps
#         maps[patient_id] = patient_maps
#     return maps

In [5]:
data = pd.read_csv('00_Data/train_scores.csv')

# Nifti MASK image
brain_mask = nb.load('00_Data/fMRI_mask.nii')

# load functional componets correlation matrix
fnc_10 = pd.read_csv('00_Data/fnc.csv')

# retrive unique network names with corresponding components
fnc10_cols = fnc_10.columns.to_list()[1:]
fnc10_cols_filtered = []
for i in fnc10_cols:
    fnc10_cols_filtered.append(i.split('_')[0])
    fnc10_cols_filtered.append(i.split('_')[2])
    
# Network index:
ntw_components = {}
ntw_labs = np.unique([i[:3] for i in fnc10_cols_filtered])
for ii in ntw_labs:
    ntw_components[ii] = np.unique([np.int(i.split('(')[-1].split(')')[0]) for i in fnc10_cols_filtered if ii in i])
    
# Look up matrix index
icn_number = pd.read_csv('00_Data/ICN_numbers.csv')

# create dictionary of networks with corresponding components
icn_idx = {}
for jj in ntw_components.keys():
    icn_idx[jj] = np.array(icn_number.index[icn_number['ICN_number'].isin(ntw_components[jj])])

In [6]:
ntw_components

{'ADN': array([21, 56]),
 'CBN': array([ 4,  7, 13, 18]),
 'CON': array([33, 37, 38, 43, 48, 55, 61, 63, 67, 68, 70, 79, 81, 83, 84, 88, 96]),
 'DMN': array([17, 23, 32, 40, 51, 71, 94]),
 'SCN': array([45, 53, 69, 98, 99]),
 'SMN': array([ 2,  3,  9, 11, 27, 54, 66, 72, 80]),
 'VSN': array([ 5,  8, 12, 15, 16, 20, 62, 77, 93])}

In [7]:
for i, ID in enumerate([1,5,6,7]):
    print(i,ID)

0 1
1 5
2 6
3 7


In [8]:
target = 'age'

train, test = model_selection.train_test_split(data, test_size=0.1)
train, val = model_selection.train_test_split(train, test_size=0.2)
X_train = train['Id']
X_val = val['Id']
X_test = test['Id']
Y_train = train[target]
Y_val = val[target]
Y_test = test[target]

In [9]:
def load_individual_maps(patient_id, icn_idx, ntwk_idx):
    patient_SM = h5py.File('00_Data/fMRI_train/{0}.mat'.format(patient_id), mode='r')
    patient_SM = np.array(patient_SM.get('SM_feature'))
    
    patient_maps = {}
    for i in icn_idx.keys():
        component_maps = {}
        for j, _ in enumerate(icn_idx[i]):
            jj = ntwk_idx[i][j]
            component_maps[jj] = patient_SM[icn_idx[i]][j].transpose([2, 1, 0])
        patient_maps[i] = component_maps
    return patient_maps

In [10]:
batch_input = ['10005', '10007', '10008']

n_blocks = 0
for i in ntwk_idx.values():
    n_blocks += len(i)

sample_shape0 = (53,63,52) # MRI map shape

k = 2 # define region split factor
channel_size = n_blocks * k**3 # corresponding channel size
# print(n_blocks, channel_size)

inputs_ = []
for i in batch_input:
    sample_maps = load_individual_maps(i, icn_idx, ntwk_idx)
    
    arr_inputs = []
    for ntw in network_names:
        for c in ntwk_idx[ntw]:
            sample_map = sample_maps[ntw][c]
            
            # padding MRI map
            shape_pad = ((sample_shape0[0]//k + 1)*k - sample_shape0[0],
                         (sample_shape0[1]//k + 1)*k - sample_shape0[1],
                         (sample_shape0[2]//k + 1)*k - sample_shape0[2])

            npad = ((shape_pad[0]//2, (shape_pad[0]//2 if shape_pad[0]%2==0 else shape_pad[0]//2+1)),    
                    (shape_pad[1]//2, (shape_pad[1]//2 if shape_pad[1]%2==0 else shape_pad[1]//2+1)),    
                    (shape_pad[2]//2, (shape_pad[2]//2 if shape_pad[2]%2==0 else shape_pad[2]//2+1)))

            sample_map_padded = np.pad(sample_map, pad_width=npad, mode='constant', constant_values=0)
            
            sx = sample_map_padded.shape[0] / k
            sy = sample_map_padded.shape[1] / k
            sz = sample_map_padded.shape[2] / k
            for kz in range(k):
                for ky in range(k):
                    for kx in range(k):
                        ki_slice = sample_map_padded[int(kx*sx): int(kx*sx + sx - 1), 
                                                     int(ky*sy): int(ky*sy + sy - 1), 
                                                     int(kz*sz): int(kz*sz + sz - 1)]
                        arr_inputs.append(ki_slice)
    arr_inputs = np.stack(arr_inputs, axis=3)
    
    inputs_.append(arr_inputs)

NameError: name 'ntwk_idx' is not defined

In [ ]:
inputs_[0].shape

In [ ]:
ntwk_idx

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

In [15]:
ds = ds.shuffle(len(X_train))

In [81]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, ntw_labs, ntw_components, icn_idx, mri_map_shape=(53,63,52), k=2, batch_size=32, shuffle=True):
        'Initialization'
# #         self.dim = dim
        self.X = list(data['Id'])
        self.y = list(data['age'])
        self.data = data
        self.batch_size = batch_size
#         self.labels = labels
#         self.list_IDs = list_IDs
#         self.n_channels = n_channels
#         self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
#         self.data = pd.read_csv('00_Data/train_scores.csv')
#         self.brain_mask = nb.load('00_Data/fMRI_mask.nii')
#         self.fnc_10 = pd.read_csv('00_Data/fnc.csv')
#         self.icn_number = pd.read_csv('00_Data/ICN_numbers.csv')
        self.ntw_labs = ntw_labs
        self.ntw_components = ntw_components
        self.icn_idx = icn_idx
        self.sample_shape0 = mri_map_shape
        self.k = k
        
    def __load_individual_maps(self, patient_id, icn_idx, ntw_components):
        patient_SM = h5py.File('00_Data/fMRI_train/{0}.mat'.format(patient_id), mode='r')
        patient_SM = np.array(patient_SM.get('SM_feature'))

        patient_maps = {}
        for i in icn_idx.keys():
            component_maps = {}
            for j, _ in enumerate(icn_idx[i]):
                jj = ntw_components[i][j]
                component_maps[jj] = patient_SM[icn_idx[i]][j].transpose([2, 1, 0])
            patient_maps[i] = component_maps
        return patient_maps
        

    def __len__(self):
        'Denotes the number of batches per epoch'
#         return int(np.floor(len(self.X) / self.batch_size))
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        X_batch = [self.X[i] for i in indexes]
        y_batch = [self.y[i] for i in indexes]
        
#         print(X_batch)

        # Generate data
        X_batch = self.__data_generation(X_batch)

        return X_batch, np.array(y_batch)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_input):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # iterate through subjects in the batch
        X = []
        for i in batch_input:
            sample_maps = self.__load_individual_maps(i, self.icn_idx, self.ntw_components)

            # iterate through mri maps of the subject
            # split each map by k**3 regions and concatenate these regions along channel dimension
            arr_regions = []
            for ntw in self.ntw_labs:
                for c in self.ntw_components[ntw]:
                    sample_map = sample_maps[ntw][c]

                    # padding MRI map
                    shape_pad = ((self.sample_shape0[0]//k + 1)*k - self.sample_shape0[0],
                                 (self.sample_shape0[1]//k + 1)*k - self.sample_shape0[1],
                                 (self.sample_shape0[2]//k + 1)*k - self.sample_shape0[2])

                    npad = ((shape_pad[0]//2, (shape_pad[0]//2 if shape_pad[0]%2==0 else shape_pad[0]//2+1)),    
                            (shape_pad[1]//2, (shape_pad[1]//2 if shape_pad[1]%2==0 else shape_pad[1]//2+1)),    
                            (shape_pad[2]//2, (shape_pad[2]//2 if shape_pad[2]%2==0 else shape_pad[2]//2+1)))

                    sample_map_padded = np.pad(sample_map, pad_width=npad, mode='constant', constant_values=0)

                    sx = sample_map_padded.shape[0] / k
                    sy = sample_map_padded.shape[1] / k
                    sz = sample_map_padded.shape[2] / k
                    for kz in range(k):
                        for ky in range(k):
                            for kx in range(k):
                                ki_region = sample_map_padded[int(kx*sx): int(kx*sx + sx - 1), 
                                                             int(ky*sy): int(ky*sy + sy - 1), 
                                                             int(kz*sz): int(kz*sz + sz - 1)]
                                arr_regions.append(ki_region)
            arr_regions = np.stack(arr_regions, axis=3)
            X.append(arr_regions)
        return np.array(X)

In [84]:
# model
model = keras.Sequential()

# convolution block #1
model.add(keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1,1,1), activation='elu', input_shape=(26, 31, 26, 424), data_format="channels_last"))
# model.add(keras.layers.Activation(keras.activations.elu(alpha=1.0)))

model.add(keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1,1,1), activation='elu'))
# model.add(keras.layers.Activation(keras.activations.elu(alpha=1.0)))

model.add(keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1)))

model.add(keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                          beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros',
                                          moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                                          beta_constraint=None, gamma_constraint=None))

# convolution block #2
model.add(keras.layers.Conv3D(32, kernel_size=(3, 3, 3), strides=(1,1,1), activation='elu'))
# model.add(keras.layers.Activation(keras.activations.elu(alpha=1.0)))

model.add(keras.layers.Conv3D(32, kernel_size=(3, 3, 3), strides=(1,1,1), activation='elu'))
# model.add(keras.layers.Activation(keras.activations.elu(alpha=1.0)))

model.add(keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1)))

model.add(keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                          beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros',
                                          moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                                          beta_constraint=None, gamma_constraint=None))

# convolution block #3
model.add(keras.layers.Conv3D(16, kernel_size=(3, 3, 3), strides=(1,1,1), activation='elu'))
# model.add(keras.layers.Activation(keras.activations.elu(alpha=1.0)))

model.add(keras.layers.Conv3D(16, kernel_size=(3, 3, 3), strides=(1,1,1), activation='elu'))
# model.add(keras.layers.Activation(keras.activations.elu(alpha=1.0)))

model.add(keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1)))

model.add(keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                          beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros',
                                          moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                                          beta_constraint=None, gamma_constraint=None))

# convolution block #4
model.add(keras.layers.Conv3D(8, kernel_size=(3, 3, 3), strides=(1,1,1), activation='elu'))
# model.add(keras.layers.Activation(keras.activations.elu(alpha=1.0)))

model.add(keras.layers.Conv3D(8, kernel_size=(3, 3, 3), strides=(1,1,1), activation='elu'))
# model.add(keras.layers.Activation(keras.activations.elu(alpha=1.0)))

model.add(keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1)))

model.add(keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                          beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros',
                                          moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                                          beta_constraint=None, gamma_constraint=None))


# hidden layer
model.add(keras.layers.Dense(256, activation='elu'))
# model.add(keras.layers.Activation(keras.activations.elu(alpha=1.0)))

# output
model.add(keras.layers.Dense(1, activation='softmax'))

In [98]:
optim = keras.optimizers.Adam(lr=0.0001,
                                 beta_1=0.99,
                                 beta_2=0.999,
                                 amsgrad=False)
        
METRICS = [keras.metrics.RootMeanSquaredError(name='rmse'),
           keras.metrics.MeanSquaredError(name='mse'),
           keras.metrics.MeanAbsoluteError(name='mae')]

In [99]:
model.compile(loss='mean_squared_error', metrics=METRICS, optimizer=optim)

In [100]:
train, test = model_selection.train_test_split(data, test_size=0.1)
train, val = model_selection.train_test_split(train, test_size=0.2)

# data, ntw_labs, ntw_components, icn_idx, mri_map_shape=(53,63,52), k=2, batch_size=32, shuffle=True

params = {'ntw_labs': ntw_labs,
          'ntw_components': ntw_components,
          'icn_idx': icn_idx,
          'mri_map_shape': (53,63,52),
          'k': 2,
          'batch_size': 5,
          'shuffle': False}

training_generator = DataGenerator(train, **params)
validation_generator = DataGenerator(val, **params)

In [96]:
X, y = training_generator.__getitem__(0)

In [102]:
X.shape

(5, 26, 31, 26, 424)

In [97]:
y.shape

(5,)

In [101]:
hist = model.fit(training_generator,
                 validation_data=validation_generator,
                 epochs=10,
                 verbose=1)

Epoch 1/10


InvalidArgumentError:  Incompatible shapes: [5,6,11,6,1] vs. [5,1]
	 [[node sub (defined at <ipython-input-101-1b74f4d93c71>:1) ]] [Op:__inference_train_function_11676]

Function call stack:
train_function


In [ ]:
results = model.evaluate(X_test, Y_test, verbose=1)

In [ ]:
    fig = plt.figure()
    fig.set_size_inches(16,16)

    ax=fig.add_subplot(3,2,1)
    ax.plot(hist.history['AUC'])
    ax.plot(hist.history['CategoricalCrossentropy'])
    ax.legend(['Metric', 'Loss'])
    ax.set_title('Train')

    ax=fig.add_subplot(3,2,2)
    ax.plot(hist.history['val_AUC'])
    ax.plot(hist.history['val_CategoricalCrossentropy'])
    ax.legend(['Metric', 'Loss'])
    ax.set_title('Test')

    ax=fig.add_subplot(3,2,3)
    ax.plot(hist.history['CategoricalCrossentropy'])
    ax.plot(hist.history['val_CategoricalCrossentropy'])
    ax.legend(['Train', 'Test'])
    ax.set_title('Loss')

    ax=fig.add_subplot(3,2,4)
    ax.plot(hist.history['AUC'])
    ax.plot(hist.history['val_AUC'])
    ax.legend(['Train', 'Test'])
    ax.set_title('Metric')